create catalog and schema

In [0]:
# %sql
# create catalog cars_catalog;

In [0]:
# %sql
# create schema cars_catalog.silver_schema;  

data reading 

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('parquet')\
    .option('inferSchema', True)\
    .load('abfss://bronze@datastorager.dfs.core.windows.net/raw_data')

In [0]:
display(df)

data transformation

In [0]:
df_trans = df.withColumn('Model_category',split(col('Model_ID'),'-')[0])

df_trans.display()


In [0]:
df.withColumn("Units_sold",col("Units_Sold").cast(StringType())).display()

In [0]:
df_trans = df_trans.withColumn("RevenuePerUnit",col("Revenue")/col("Units_Sold"))
df.display()

In [0]:
df.groupBy("Date_ID","BranchName").agg(sum('Units_Sold').alias("total_units_sold")).sort("Date_ID","total_units_sold",ascending=[True, False]).display()


Databricks visualization. Run in Databricks to view.

In [0]:
df_trans.write.format("parquet").mode("overwrite").option('path','abfss://silver@datastorager.dfs.core.windows.net/car_sales').save()

In [0]:
%sql
select * from parquet.`abfss://silver@datastorager.dfs.core.windows.net/car_sales`